# A2.1 Regresión Logística y Validación Cruzada

**Autor:** Ricardo Arath Sanchez Aguirre

**Materia:** SC3314 – Inteligencia Artificial  
**Universidad:** Universidad de Monterrey  
**Profesor:** Dr. Antonio Martínez Torteya  

---

## Índice
1. [Introducción](#1-introducción)
2. [Fundamentos Teóricos](#2-fundamentos-teóricos)
3. [Carga y Exploración de Datos](#3-carga-y-exploración-de-datos)
4. [Preparación de Datos](#4-preparación-de-datos)
5. [Implementación de Regresión Logística](#5-implementación-de-regresión-logística)
6. [Validación Cruzada](#6-validación-cruzada)
7. [Evaluación del Modelo](#7-evaluación-del-modelo)
8. [Conclusiones](#8-conclusiones)

---

# 1. Introducción

## 1.1 Contexto del Análisis

En este análisis utilizamos la **Estadística de Matrimonios 2024** del INEGI para implementar un modelo de **Regresión Logística**, uno de los algoritmos fundamentales del aprendizaje supervisado para problemas de clasificación.

## 1.2 Problema de Clasificación

**Objetivo:** Predecir el **régimen matrimonial** (Sociedad Conyugal vs Separación de Bienes) basándose en las características sociodemográficas de los contrayentes.

### ¿Por qué este problema?

El régimen matrimonial es una decisión importante que refleja:
- **Nivel socioeconómico**: Personas con mayor patrimonio tienden a elegir separación de bienes
- **Edad de los contrayentes**: Segundas nupcias suelen preferir separación de bienes
- **Nivel educativo**: Mayor educación puede asociarse con mayor conciencia legal
- **Contexto geográfico**: Diferencias culturales entre regiones

## 1.3 Variables de Interés

| Variable | Tipo | Descripción |
|----------|------|-------------|
| **regimen_ma** (objetivo) | Binaria | 1=Sociedad Conyugal, 2=Separación de Bienes |
| edad_con1, edad_con2 | Numérica | Edades de los contrayentes |
| escol_con1, escol_con2 | Ordinal | Nivel de escolaridad |
| tam_loc_re | Ordinal | Tamaño de localidad |
| ent_regis | Categórica | Entidad federativa |

# 2. Fundamentos Teóricos

## 2.1 ¿Qué es la Regresión Logística?

La **Regresión Logística** es un algoritmo de clasificación supervisada que modela la probabilidad de que una observación pertenezca a una clase particular.

### Diferencias con Regresión Lineal

| Aspecto | Regresión Lineal | Regresión Logística |
|---------|------------------|--------------------|
| **Variable objetivo** | Continua | Categórica (binaria) |
| **Predicción** | Valor numérico | Probabilidad [0, 1] |
| **Función** | $f(x) = wx + b$ | $f(x) = \sigma(wx + b)$ |
| **Función de costo** | MSE | Cross-Entropy (Log Loss) |

## 2.2 La Función Sigmoide

La **función sigmoide** (o logística) transforma cualquier valor real a un rango entre 0 y 1:

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

Donde $z = \mathbf{w} \cdot \mathbf{x} + b$ (combinación lineal de las características).

### Propiedades de la función sigmoide:
- **Rango**: $(0, 1)$ - ideal para representar probabilidades
- **Punto medio**: $\sigma(0) = 0.5$
- **Asintótica**: Se aproxima a 0 cuando $z \to -\infty$ y a 1 cuando $z \to +\infty$
- **Derivada**: $\sigma'(z) = \sigma(z)(1 - \sigma(z))$ - facilita el cálculo del gradiente

## 2.3 Función de Costo (Log Loss)

La función de costo para regresión logística es la **entropía cruzada binaria** (Binary Cross-Entropy):

$$J(\mathbf{w}, b) = -\frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log(\hat{y}^{(i)}) + (1 - y^{(i)}) \log(1 - \hat{y}^{(i)}) \right]$$

Donde:
- $m$ = número de muestras
- $y^{(i)}$ = etiqueta real (0 o 1)
- $\hat{y}^{(i)} = \sigma(\mathbf{w} \cdot \mathbf{x}^{(i)} + b)$ = probabilidad predicha

### ¿Por qué Log Loss y no MSE?
- MSE en clasificación produce una superficie de costo **no convexa** con múltiples mínimos locales
- Log Loss es **convexa**, garantizando un único mínimo global
- Penaliza más fuertemente las predicciones muy seguras pero incorrectas

## 2.4 Descenso del Gradiente

Los parámetros se actualizan iterativamente:

$$w_j := w_j - \alpha \frac{\partial J}{\partial w_j}$$

$$b := b - \alpha \frac{\partial J}{\partial b}$$

Donde:
$$\frac{\partial J}{\partial w_j} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)}) x_j^{(i)}$$

$$\frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})$$

## 2.5 Validación Cruzada (Cross-Validation)

La **validación cruzada K-Fold** es una técnica para evaluar la capacidad de generalización del modelo:

1. Divide los datos en K particiones (folds) de tamaño similar
2. Para cada fold $k$:
   - Usa fold $k$ como conjunto de validación
   - Usa los otros $K-1$ folds como entrenamiento
   - Evalúa el modelo en el fold de validación
3. Promedia las métricas de los K experimentos

### Ventajas de K-Fold:
- Usa **todos los datos** para entrenamiento y validación
- Proporciona una estimación más **robusta** del desempeño
- Reduce la varianza en la estimación del error

# 3. Carga y Exploración de Datos

## 3.1 Importación de Librerías

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Modelos y métricas de Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, classification_report, roc_curve, auc,
                             roc_auc_score, log_loss)

print("Librerías importadas correctamente")

## 3.2 Carga del Dataset

In [ ]:
# Cargar el conjunto de datos de matrimonios 2024
ruta_datos = 'conjunto_de_datos/conjunto_de_datos_emat2024.csv'
df = pd.read_csv(ruta_datos)

print(f"Dimensiones del dataset: {df.shape}")
print(f"Total de matrimonios registrados: {len(df):,}")
print(f"\nColumnas disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

In [ ]:
# Visualizar las primeras filas
df.head()

## 3.3 Análisis de la Variable Objetivo

In [ ]:
# Analizar la distribución del régimen matrimonial
print("Distribución del Régimen Matrimonial:")
print("="*50)
regimen_counts = df['regimen_ma'].value_counts().sort_index()

regimen_map = {
    1: 'Sociedad Conyugal',
    2: 'Separación de Bienes',
    3: 'Mixto',
    9: 'No Especificado'
}

for codigo, count in regimen_counts.items():
    nombre = regimen_map.get(codigo, f'Código {codigo}')
    porcentaje = count / len(df) * 100
    print(f"{codigo}: {nombre:25s} -> {count:>8,} ({porcentaje:5.2f}%)")

In [ ]:
# Visualización de la distribución
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras
colors = ['steelblue', 'coral', 'lightgreen', 'gray']
regimen_labels = [regimen_map.get(r, str(r)) for r in regimen_counts.index]
bars = axes[0].bar(regimen_labels, regimen_counts.values, color=colors[:len(regimen_counts)], edgecolor='black')
axes[0].set_xlabel('Régimen Matrimonial', fontsize=12)
axes[0].set_ylabel('Frecuencia', fontsize=12)
axes[0].set_title('Distribución del Régimen Matrimonial', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=15)

# Añadir etiquetas de valor
for bar, count in zip(bars, regimen_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1000,
                 f'{count:,}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Gráfico de pastel (solo para régimen válido: 1 y 2)
df_valido = df[df['regimen_ma'].isin([1, 2])]
regimen_valido = df_valido['regimen_ma'].value_counts()
labels_pie = ['Sociedad Conyugal', 'Separación de Bienes']
explode = (0.02, 0.02)
axes[1].pie(regimen_valido.values, labels=labels_pie, autopct='%1.1f%%',
            colors=['steelblue', 'coral'], explode=explode, startangle=90,
            wedgeprops={'edgecolor': 'black', 'linewidth': 1})
axes[1].set_title('Proporción de Régimen Matrimonial\n(Solo casos válidos)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nNota: Para el modelo de clasificación binaria, usaremos solo los casos")
print(f"con Sociedad Conyugal (1) y Separación de Bienes (2).")
print(f"Esto representa {len(df_valido):,} registros ({len(df_valido)/len(df)*100:.1f}% del total).")

# 4. Preparación de Datos

## 4.1 Filtrado y Limpieza

In [ ]:
# Crear copia del dataframe para el modelo
df_modelo = df.copy()
registros_iniciales = len(df_modelo)

# 1. Filtrar solo regímenes válidos (1=Sociedad Conyugal, 2=Separación de Bienes)
df_modelo = df_modelo[df_modelo['regimen_ma'].isin([1, 2])]
print(f"Después de filtrar régimen válido: {len(df_modelo):,} registros")

# 2. Eliminar edades no especificadas (código 99)
df_modelo = df_modelo[(df_modelo['edad_con1'] != 99) & (df_modelo['edad_con2'] != 99)]
print(f"Después de filtrar edades válidas: {len(df_modelo):,} registros")

# 3. Filtrar escolaridades válidas (excluir 9=No especificado)
df_modelo = df_modelo[(df_modelo['escol_con1'] < 9) & (df_modelo['escol_con2'] < 9)]
print(f"Después de filtrar escolaridad válida: {len(df_modelo):,} registros")

# 4. Filtrar tamaño de localidad válido (excluir 99=No especificado)
df_modelo = df_modelo[df_modelo['tam_loc_re'] != 99]
print(f"Después de filtrar localidad válida: {len(df_modelo):,} registros")

print(f"\nRegistros finales: {len(df_modelo):,} ({len(df_modelo)/registros_iniciales*100:.1f}% del total)")

## 4.2 Creación de la Variable Objetivo Binaria

In [ ]:
# Crear variable objetivo binaria
# 0 = Sociedad Conyugal (código 1)
# 1 = Separación de Bienes (código 2)

df_modelo['y_regimen'] = (df_modelo['regimen_ma'] == 2).astype(int)

print("Variable objetivo creada:")
print("="*50)
print(f"0 (Sociedad Conyugal):     {(df_modelo['y_regimen'] == 0).sum():>10,} ({(df_modelo['y_regimen'] == 0).mean()*100:.2f}%)")
print(f"1 (Separación de Bienes):  {(df_modelo['y_regimen'] == 1).sum():>10,} ({(df_modelo['y_regimen'] == 1).mean()*100:.2f}%)")

# Verificar balance de clases
ratio = df_modelo['y_regimen'].mean()
print(f"\nRatio de clase positiva: {ratio:.2%}")
if 0.3 <= ratio <= 0.7:
    print("Las clases están relativamente balanceadas.")
else:
    print("ADVERTENCIA: Desbalance de clases detectado. Considerar técnicas de balanceo.")

## 4.3 Ingeniería de Características

In [ ]:
# Crear nuevas características derivadas
print("Creando nuevas características...")
print("="*50)

# 1. Promedio de edades
df_modelo['edad_promedio'] = (df_modelo['edad_con1'] + df_modelo['edad_con2']) / 2
print("- edad_promedio: Promedio de edades de ambos contrayentes")

# 2. Diferencia de edad (valor absoluto)
df_modelo['diferencia_edad'] = np.abs(df_modelo['edad_con1'] - df_modelo['edad_con2'])
print("- diferencia_edad: Diferencia absoluta de edades")

# 3. Promedio de escolaridad
df_modelo['escol_promedio'] = (df_modelo['escol_con1'] + df_modelo['escol_con2']) / 2
print("- escol_promedio: Promedio de escolaridad de ambos contrayentes")

# 4. Diferencia de escolaridad (valor absoluto)
df_modelo['diferencia_escol'] = np.abs(df_modelo['escol_con1'] - df_modelo['escol_con2'])
print("- diferencia_escol: Diferencia absoluta de escolaridad")

# 5. Indicador de matrimonio del mismo sexo
df_modelo['mismo_sexo'] = (df_modelo['genero'] == 2).astype(int)
print("- mismo_sexo: Indicador de matrimonio del mismo sexo")

# 6. Indicador de localidad urbana (>=50,000 habitantes)
df_modelo['es_urbano'] = (df_modelo['tam_loc_re'] >= 11).astype(int)
print("- es_urbano: Indicador de localidad urbana (>=50,000 hab)")

# 7. Indicador de alta escolaridad (al menos uno con nivel profesional)
df_modelo['alta_escolaridad'] = ((df_modelo['escol_con1'] >= 7) | (df_modelo['escol_con2'] >= 7)).astype(int)
print("- alta_escolaridad: Al menos un contrayente con nivel profesional")

# 8. Indicador de matrimonio tardío (ambos >35 años)
df_modelo['matrimonio_tardio'] = ((df_modelo['edad_con1'] > 35) & (df_modelo['edad_con2'] > 35)).astype(int)
print("- matrimonio_tardio: Ambos contrayentes mayores de 35 años")

print(f"\nTotal de características disponibles: {df_modelo.shape[1]}")

## 4.4 Selección de Características para el Modelo

In [ ]:
# Seleccionar características para el modelo
features = [
    'edad_con1',           # Edad del primer contrayente
    'edad_con2',           # Edad del segundo contrayente
    'edad_promedio',       # Promedio de edades
    'diferencia_edad',     # Diferencia de edades
    'escol_con1',          # Escolaridad contrayente 1
    'escol_con2',          # Escolaridad contrayente 2
    'escol_promedio',      # Promedio de escolaridad
    'diferencia_escol',    # Diferencia de escolaridad
    'tam_loc_re',          # Tamaño de localidad
    'es_urbano',           # Es localidad urbana
    'mismo_sexo',          # Matrimonio del mismo sexo
    'alta_escolaridad',    # Alta escolaridad
    'matrimonio_tardio'    # Matrimonio tardío
]

print(f"Características seleccionadas: {len(features)}")
for i, f in enumerate(features, 1):
    print(f"  {i:2d}. {f}")

In [ ]:
# Análisis de correlación con la variable objetivo
correlaciones = df_modelo[features + ['y_regimen']].corr()['y_regimen'].drop('y_regimen').sort_values(key=abs, ascending=False)

print("Correlación de características con el Régimen Matrimonial:")
print("="*60)
print(f"{'Variable':25s} {'Correlación':>15s} {'Dirección':>15s}")
print("-"*60)
for var, corr in correlaciones.items():
    direccion = 'Positiva (+)' if corr > 0 else 'Negativa (-)'
    print(f"{var:25s} {corr:+15.4f} {direccion:>15s}")

In [ ]:
# Visualización de correlaciones
fig, ax = plt.subplots(figsize=(10, 6))
correlaciones_sorted = correlaciones.sort_values(ascending=True)
colors = ['green' if x > 0 else 'red' for x in correlaciones_sorted.values]
correlaciones_sorted.plot(kind='barh', color=colors, edgecolor='black', ax=ax)
ax.set_xlabel('Correlación con Régimen Matrimonial', fontsize=12)
ax.set_title('Correlación de Variables con Separación de Bienes (vs Sociedad Conyugal)',
             fontsize=14, fontweight='bold')
ax.axvline(x=0, color='black', linewidth=0.5)
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 4.5 Preparación de Datos para el Modelo

In [ ]:
# Preparar matrices X e y
X = df_modelo[features].copy()
y = df_modelo['y_regimen'].copy()

print(f"Dimensiones de X: {X.shape}")
print(f"Dimensiones de y: {y.shape}")
print(f"\nValores nulos en X: {X.isnull().sum().sum()}")
print(f"Valores nulos en y: {y.isnull().sum()}")

In [ ]:
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("División de datos:")
print("="*50)
print(f"Conjunto de entrenamiento: {len(X_train):,} registros ({len(X_train)/len(X)*100:.1f}%)")
print(f"Conjunto de prueba:        {len(X_test):,} registros ({len(X_test)/len(X)*100:.1f}%)")
print(f"\nDistribución de clases en entrenamiento:")
print(f"  Clase 0 (Sociedad Conyugal):    {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.2f}%)")
print(f"  Clase 1 (Separación Bienes):    {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.2f}%)")
print(f"\nDistribución de clases en prueba:")
print(f"  Clase 0 (Sociedad Conyugal):    {(y_test == 0).sum():,} ({(y_test == 0).mean()*100:.2f}%)")
print(f"  Clase 1 (Separación Bienes):    {(y_test == 1).sum():,} ({(y_test == 1).mean()*100:.2f}%)")

In [ ]:
# Estandarizar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("- Características estandarizadas (media=0, std=1)")
print(f"\nEstadísticas de entrenamiento (estandarizadas):")
print(f"  Media: {X_train_scaled.mean():.6f}")
print(f"  Std:   {X_train_scaled.std():.6f}")

# 5. Implementación de Regresión Logística

## 5.1 Implementación Manual de las Funciones Básicas

In [ ]:
# Implementación de la función sigmoide
def sigmoid(z):
    """
    Calcula la función sigmoide.
    
    Parámetros:
    -----------
    z : array-like
        Entrada (puede ser escalar o array)
    
    Retorna:
    --------
    g : array-like
        Valor de la función sigmoide
    """
    # Prevenir overflow
    z = np.clip(z, -500, 500)
    g = 1 / (1 + np.exp(-z))
    return g

# Visualizar la función sigmoide
z_values = np.linspace(-10, 10, 100)
sigmoid_values = sigmoid(z_values)

plt.figure(figsize=(10, 6))
plt.plot(z_values, sigmoid_values, 'b-', linewidth=2, label=r'$\sigma(z) = \frac{1}{1+e^{-z}}$')
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Umbral (0.5)')
plt.axvline(x=0, color='gray', linestyle=':', alpha=0.7)
plt.xlabel('z', fontsize=12)
plt.ylabel(r'$\sigma(z)$', fontsize=12)
plt.title('Función Sigmoide', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim(-0.1, 1.1)
plt.show()

print("Verificación de la función sigmoide:")
print(f"  sigmoid(0) = {sigmoid(0):.4f} (esperado: 0.5)")
print(f"  sigmoid(-5) = {sigmoid(-5):.4f} (esperado: ~0.007)")
print(f"  sigmoid(5) = {sigmoid(5):.4f} (esperado: ~0.993)")

In [ ]:
# Implementación de la función de costo (Log Loss)
def compute_cost(X, y, w, b):
    """
    Calcula la función de costo (Binary Cross-Entropy) para regresión logística.
    
    Parámetros:
    -----------
    X : array de forma (m, n)
        Matriz de características
    y : array de forma (m,)
        Etiquetas verdaderas (0 o 1)
    w : array de forma (n,)
        Pesos del modelo
    b : escalar
        Sesgo (bias) del modelo
    
    Retorna:
    --------
    cost : escalar
        Valor de la función de costo
    """
    m = len(y)
    
    # Calcular predicciones
    z = np.dot(X, w) + b
    y_hat = sigmoid(z)
    
    # Evitar log(0) usando clipping
    epsilon = 1e-15
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)
    
    # Calcular costo (Binary Cross-Entropy)
    cost = -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    
    return cost

# Verificar función de costo con valores iniciales
w_init = np.zeros(X_train_scaled.shape[1])
b_init = 0
cost_initial = compute_cost(X_train_scaled, y_train.values, w_init, b_init)
print(f"Costo inicial (w=0, b=0): {cost_initial:.6f}")
print(f"Costo esperado para pesos cero con clases balanceadas: ~{-np.log(0.5):.6f} = ln(2)")

In [ ]:
# Implementación del gradiente
def compute_gradient(X, y, w, b):
    """
    Calcula el gradiente de la función de costo.
    
    Parámetros:
    -----------
    X : array de forma (m, n)
        Matriz de características
    y : array de forma (m,)
        Etiquetas verdaderas
    w : array de forma (n,)
        Pesos del modelo
    b : escalar
        Sesgo del modelo
    
    Retorna:
    --------
    dj_dw : array de forma (n,)
        Gradiente respecto a w
    dj_db : escalar
        Gradiente respecto a b
    """
    m = len(y)
    
    # Calcular predicciones
    z = np.dot(X, w) + b
    y_hat = sigmoid(z)
    
    # Calcular gradientes
    error = y_hat - y
    dj_dw = (1 / m) * np.dot(X.T, error)
    dj_db = (1 / m) * np.sum(error)
    
    return dj_dw, dj_db

# Verificar gradiente
dj_dw, dj_db = compute_gradient(X_train_scaled, y_train.values, w_init, b_init)
print(f"Gradientes iniciales:")
print(f"  dJ/db = {dj_db:.6f}")
print(f"  dJ/dw (primeras 5): {dj_dw[:5]}")

In [ ]:
# Implementación del descenso del gradiente
def gradient_descent(X, y, w_init, b_init, alpha, num_iters, verbose=True):
    """
    Ejecuta el descenso del gradiente para optimizar los parámetros.
    
    Parámetros:
    -----------
    X : array de forma (m, n)
        Matriz de características
    y : array de forma (m,)
        Etiquetas verdaderas
    w_init : array de forma (n,)
        Pesos iniciales
    b_init : escalar
        Sesgo inicial
    alpha : escalar
        Tasa de aprendizaje
    num_iters : int
        Número de iteraciones
    verbose : bool
        Si se muestra progreso
    
    Retorna:
    --------
    w : array de forma (n,)
        Pesos optimizados
    b : escalar
        Sesgo optimizado
    cost_history : lista
        Historia de costos
    """
    w = w_init.copy()
    b = b_init
    cost_history = []
    
    for i in range(num_iters):
        # Calcular gradientes
        dj_dw, dj_db = compute_gradient(X, y, w, b)
        
        # Actualizar parámetros
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        # Guardar costo
        cost = compute_cost(X, y, w, b)
        cost_history.append(cost)
        
        # Mostrar progreso
        if verbose and i % (num_iters // 10) == 0:
            print(f"  Iteración {i:5d}: Costo = {cost:.6f}")
    
    return w, b, cost_history

print("Función de descenso del gradiente implementada.")

## 5.2 Entrenamiento del Modelo Manual

In [ ]:
# Entrenar modelo con implementación manual
print("ENTRENAMIENTO DEL MODELO (Implementación Manual)")
print("="*60)

# Hiperparámetros
alpha = 0.1          # Tasa de aprendizaje
num_iters = 1000     # Número de iteraciones

# Inicializar parámetros
w_init = np.zeros(X_train_scaled.shape[1])
b_init = 0

print(f"Hiperparámetros:")
print(f"  Learning rate (α): {alpha}")
print(f"  Iteraciones: {num_iters}")
print(f"\nProgreso del entrenamiento:")

# Ejecutar descenso del gradiente
w_manual, b_manual, cost_history = gradient_descent(
    X_train_scaled, y_train.values, w_init, b_init, alpha, num_iters
)

print(f"\nEntrenamiento completado.")
print(f"Costo final: {cost_history[-1]:.6f}")

In [ ]:
# Visualizar convergencia
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Curva de costo completa
axes[0].plot(cost_history, 'b-', linewidth=1.5)
axes[0].set_xlabel('Iteración', fontsize=12)
axes[0].set_ylabel('Costo (Log Loss)', fontsize=12)
axes[0].set_title('Convergencia del Descenso del Gradiente', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Gráfico 2: Primeras 200 iteraciones (zoom)
axes[1].plot(cost_history[:200], 'b-', linewidth=1.5)
axes[1].set_xlabel('Iteración', fontsize=12)
axes[1].set_ylabel('Costo (Log Loss)', fontsize=12)
axes[1].set_title('Primeras 200 Iteraciones (Zoom)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Reducción del costo: {cost_history[0]:.6f} → {cost_history[-1]:.6f}")
print(f"Reducción porcentual: {(1 - cost_history[-1]/cost_history[0])*100:.2f}%")

In [ ]:
# Mostrar coeficientes del modelo manual
print("Coeficientes del Modelo (Implementación Manual):")
print("="*60)
print(f"Intercepto (b): {b_manual:.6f}")
print(f"\nPesos (w):")

coef_df = pd.DataFrame({
    'Variable': features,
    'Coeficiente': w_manual,
    'Coef. Absoluto': np.abs(w_manual)
}).sort_values('Coef. Absoluto', ascending=False)

for _, row in coef_df.iterrows():
    signo = '+' if row['Coeficiente'] > 0 else ''
    print(f"  {row['Variable']:25s}: {signo}{row['Coeficiente']:.6f}")

## 5.3 Modelo con Scikit-Learn (Comparación)

In [ ]:
# Entrenar modelo con Scikit-Learn para comparación
print("MODELO CON SCIKIT-LEARN (Comparación)")
print("="*60)

modelo_sklearn = LogisticRegression(
    max_iter=1000,
    solver='lbfgs',
    random_state=42
)

modelo_sklearn.fit(X_train_scaled, y_train)

print(f"Intercepto (sklearn): {modelo_sklearn.intercept_[0]:.6f}")
print(f"Intercepto (manual):  {b_manual:.6f}")
print(f"\nComparación de coeficientes:")
print(f"{'Variable':25s} {'Sklearn':>12s} {'Manual':>12s} {'Diferencia':>12s}")
print("-"*65)
for i, feat in enumerate(features):
    w_sk = modelo_sklearn.coef_[0][i]
    w_mn = w_manual[i]
    diff = abs(w_sk - w_mn)
    print(f"{feat:25s} {w_sk:12.6f} {w_mn:12.6f} {diff:12.6f}")

# 6. Validación Cruzada

## 6.1 Implementación de K-Fold Cross-Validation

In [ ]:
# Implementación de validación cruzada K-Fold
print("VALIDACIÓN CRUZADA K-FOLD")
print("="*60)

# Configurar K-Fold estratificado
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

print(f"Número de folds: {k_folds}")
print(f"\nEjecutando validación cruzada...\n")

# Almacenar resultados de cada fold
fold_results = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'auc': [],
    'log_loss': []
}

# Ejecutar validación cruzada manualmente para tener más control
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    # Dividir datos
    X_train_fold = X.iloc[train_idx]
    X_val_fold = X.iloc[val_idx]
    y_train_fold = y.iloc[train_idx]
    y_val_fold = y.iloc[val_idx]
    
    # Escalar
    scaler_fold = StandardScaler()
    X_train_fold_scaled = scaler_fold.fit_transform(X_train_fold)
    X_val_fold_scaled = scaler_fold.transform(X_val_fold)
    
    # Entrenar modelo
    modelo_fold = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42)
    modelo_fold.fit(X_train_fold_scaled, y_train_fold)
    
    # Predicciones
    y_pred_fold = modelo_fold.predict(X_val_fold_scaled)
    y_prob_fold = modelo_fold.predict_proba(X_val_fold_scaled)[:, 1]
    
    # Calcular métricas
    fold_results['accuracy'].append(accuracy_score(y_val_fold, y_pred_fold))
    fold_results['precision'].append(precision_score(y_val_fold, y_pred_fold))
    fold_results['recall'].append(recall_score(y_val_fold, y_pred_fold))
    fold_results['f1'].append(f1_score(y_val_fold, y_pred_fold))
    fold_results['auc'].append(roc_auc_score(y_val_fold, y_prob_fold))
    fold_results['log_loss'].append(log_loss(y_val_fold, y_prob_fold))
    
    print(f"Fold {fold}: Accuracy={fold_results['accuracy'][-1]:.4f}, "
          f"F1={fold_results['f1'][-1]:.4f}, AUC={fold_results['auc'][-1]:.4f}")

print(f"\n" + "="*60)
print("RESULTADOS PROMEDIO DE VALIDACIÓN CRUZADA:")
print("="*60)
for metrica, valores in fold_results.items():
    media = np.mean(valores)
    std = np.std(valores)
    print(f"{metrica:12s}: {media:.4f} ± {std:.4f}")

In [ ]:
# Visualización de resultados de validación cruzada
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico 1: Métricas por fold
metricas_plot = ['accuracy', 'precision', 'recall', 'f1']
x_pos = np.arange(k_folds)
width = 0.2

for i, metrica in enumerate(metricas_plot):
    axes[0].bar(x_pos + i*width, fold_results[metrica], width, label=metrica.capitalize())

axes[0].set_xlabel('Fold', fontsize=12)
axes[0].set_ylabel('Valor', fontsize=12)
axes[0].set_title('Métricas por Fold', fontsize=14, fontweight='bold')
axes[0].set_xticks(x_pos + 1.5*width)
axes[0].set_xticklabels([f'Fold {i+1}' for i in range(k_folds)])
axes[0].legend(loc='lower right')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim(0, 1)

# Gráfico 2: Box plots de métricas
data_boxplot = [fold_results[m] for m in metricas_plot]
bp = axes[1].boxplot(data_boxplot, labels=[m.capitalize() for m in metricas_plot], patch_artist=True)
colors_bp = ['steelblue', 'coral', 'lightgreen', 'gold']
for patch, color in zip(bp['boxes'], colors_bp):
    patch.set_facecolor(color)
axes[1].set_ylabel('Valor', fontsize=12)
axes[1].set_title('Distribución de Métricas (K-Fold CV)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6.2 Validación Cruzada con cross_val_score de Scikit-Learn

In [ ]:
# Usar cross_val_score para validación rápida
print("VALIDACIÓN CRUZADA CON cross_val_score")
print("="*60)

# Escalar todos los datos
scaler_all = StandardScaler()
X_scaled = scaler_all.fit_transform(X)

# Diferentes métricas de scoring
scoring_metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']

modelo_cv = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42)

print(f"{'Métrica':12s} {'Media':>12s} {'Std':>12s} {'Intervalo 95%':>20s}")
print("-"*60)

for scoring in scoring_metrics:
    scores = cross_val_score(modelo_cv, X_scaled, y, cv=5, scoring=scoring)
    intervalo = f"[{scores.mean() - 1.96*scores.std():.4f}, {scores.mean() + 1.96*scores.std():.4f}]"
    print(f"{scoring:12s} {scores.mean():12.4f} {scores.std():12.4f} {intervalo:>20s}")

# 7. Evaluación del Modelo

## 7.1 Predicciones en Conjunto de Prueba

In [ ]:
# Predicciones con el modelo de sklearn
y_pred_test = modelo_sklearn.predict(X_test_scaled)
y_prob_test = modelo_sklearn.predict_proba(X_test_scaled)[:, 1]

print("PREDICCIONES EN CONJUNTO DE PRUEBA")
print("="*60)
print(f"Total de predicciones: {len(y_pred_test):,}")
print(f"\nDistribución de predicciones:")
print(f"  Clase 0 (Sociedad Conyugal):    {(y_pred_test == 0).sum():,} ({(y_pred_test == 0).mean()*100:.2f}%)")
print(f"  Clase 1 (Separación Bienes):    {(y_pred_test == 1).sum():,} ({(y_pred_test == 1).mean()*100:.2f}%)")

## 7.2 Métricas de Evaluación

In [ ]:
# Calcular todas las métricas
print("MÉTRICAS DE EVALUACIÓN")
print("="*60)

accuracy = accuracy_score(y_test, y_pred_test)
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
auc_score = roc_auc_score(y_test, y_prob_test)
logloss = log_loss(y_test, y_prob_test)

print(f"{'Métrica':<20s} {'Valor':>10s} {'Descripción'}")
print("-"*70)
print(f"{'Accuracy':<20s} {accuracy:>10.4f} Proporción de predicciones correctas")
print(f"{'Precision':<20s} {precision:>10.4f} De los predichos positivos, ¿cuántos son correctos?")
print(f"{'Recall':<20s} {recall:>10.4f} De los positivos reales, ¿cuántos detectamos?")
print(f"{'F1-Score':<20s} {f1:>10.4f} Media armónica de Precision y Recall")
print(f"{'AUC-ROC':<20s} {auc_score:>10.4f} Área bajo la curva ROC")
print(f"{'Log Loss':<20s} {logloss:>10.4f} Entropía cruzada (menor es mejor)")

## 7.3 Matriz de Confusión

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_test)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matriz de confusión con valores absolutos
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Sociedad Conyugal', 'Separación Bienes'],
            yticklabels=['Sociedad Conyugal', 'Separación Bienes'])
axes[0].set_xlabel('Predicción', fontsize=12)
axes[0].set_ylabel('Valor Real', fontsize=12)
axes[0].set_title('Matriz de Confusión (Valores Absolutos)', fontsize=14, fontweight='bold')

# Matriz de confusión normalizada
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues', ax=axes[1],
            xticklabels=['Sociedad Conyugal', 'Separación Bienes'],
            yticklabels=['Sociedad Conyugal', 'Separación Bienes'])
axes[1].set_xlabel('Predicción', fontsize=12)
axes[1].set_ylabel('Valor Real', fontsize=12)
axes[1].set_title('Matriz de Confusión (Normalizada por Fila)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nInterpretación de la Matriz de Confusión:")
print(f"  Verdaderos Negativos (TN): {cm[0,0]:,} - Sociedad Conyugal correctamente predichos")
print(f"  Falsos Positivos (FP):     {cm[0,1]:,} - Sociedad Conyugal predichos como Separación")
print(f"  Falsos Negativos (FN):     {cm[1,0]:,} - Separación predichos como Sociedad Conyugal")
print(f"  Verdaderos Positivos (TP): {cm[1,1]:,} - Separación de Bienes correctamente predichos")

## 7.4 Curva ROC

In [ ]:
# Curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_prob_test)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='steelblue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - Specificidad)', fontsize=12)
plt.ylabel('True Positive Rate (Sensibilidad)', fontsize=12)
plt.title('Curva ROC - Regresión Logística', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)

# Encontrar umbral óptimo (punto más cercano a esquina superior izquierda)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], marker='o', color='red', s=100,
            label=f'Umbral óptimo = {optimal_threshold:.2f}')
plt.legend(loc='lower right', fontsize=11)

plt.tight_layout()
plt.show()

print(f"\nUmbral óptimo: {optimal_threshold:.4f}")
print(f"En este punto: TPR = {tpr[optimal_idx]:.4f}, FPR = {fpr[optimal_idx]:.4f}")

## 7.5 Importancia de Variables

In [ ]:
# Importancia de variables basada en coeficientes
coef_importance = pd.DataFrame({
    'Variable': features,
    'Coeficiente': modelo_sklearn.coef_[0],
    'Odds Ratio': np.exp(modelo_sklearn.coef_[0]),
    'Importancia (|coef|)': np.abs(modelo_sklearn.coef_[0])
}).sort_values('Importancia (|coef|)', ascending=False)

print("IMPORTANCIA DE VARIABLES")
print("="*80)
print(f"{'Variable':25s} {'Coeficiente':>12s} {'Odds Ratio':>12s} {'Interpretación'}")
print("-"*80)
for _, row in coef_importance.iterrows():
    if row['Coeficiente'] > 0:
        interp = f"↑ prob. Sep. Bienes"
    else:
        interp = f"↓ prob. Sep. Bienes"
    print(f"{row['Variable']:25s} {row['Coeficiente']:+12.4f} {row['Odds Ratio']:12.4f} {interp}")

In [ ]:
# Visualización de coeficientes
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico 1: Coeficientes
coef_sorted = coef_importance.sort_values('Coeficiente', ascending=True)
colors = ['green' if x > 0 else 'red' for x in coef_sorted['Coeficiente']]
axes[0].barh(coef_sorted['Variable'], coef_sorted['Coeficiente'], color=colors, edgecolor='black')
axes[0].set_xlabel('Coeficiente', fontsize=12)
axes[0].set_title('Coeficientes del Modelo\n(Verde = ↑Sep.Bienes, Rojo = ↓Sep.Bienes)',
                  fontsize=14, fontweight='bold')
axes[0].axvline(x=0, color='black', linewidth=0.5)
axes[0].grid(True, alpha=0.3, axis='x')

# Gráfico 2: Odds Ratios
axes[1].barh(coef_sorted['Variable'], coef_sorted['Odds Ratio'], color='steelblue', edgecolor='black')
axes[1].set_xlabel('Odds Ratio', fontsize=12)
axes[1].set_title('Odds Ratios\n(>1 = ↑probabilidad, <1 = ↓probabilidad)',
                  fontsize=14, fontweight='bold')
axes[1].axvline(x=1, color='red', linewidth=1.5, linestyle='--', label='OR = 1 (sin efecto)')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 7.6 Reporte de Clasificación

In [ ]:
# Reporte de clasificación completo
print("REPORTE DE CLASIFICACIÓN")
print("="*60)
print(classification_report(y_test, y_pred_test, 
                            target_names=['Sociedad Conyugal', 'Separación Bienes']))

# 8. Conclusiones

## 8.1 Resumen de Resultados

In [ ]:
# Resumen final
print("="*70)
print("RESUMEN FINAL DEL ANÁLISIS")
print("="*70)

print("\nDATOS:")
print(f"   • Registros totales utilizados: {len(X):,}")
print(f"   • Características: {len(features)}")
print(f"   • División: 80% entrenamiento, 20% prueba")

print("\nDESEMPEÑO DEL MODELO:")
print(f"   • Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   • Precision: {precision:.4f}")
print(f"   • Recall:    {recall:.4f}")
print(f"   • F1-Score:  {f1:.4f}")
print(f"   • AUC-ROC:   {auc_score:.4f}")

print("\nVALIDACIÓN CRUZADA (5-Fold):")
print(f"   • Accuracy promedio: {np.mean(fold_results['accuracy']):.4f} ± {np.std(fold_results['accuracy']):.4f}")
print(f"   • F1 promedio:       {np.mean(fold_results['f1']):.4f} ± {np.std(fold_results['f1']):.4f}")
print(f"   • AUC promedio:      {np.mean(fold_results['auc']):.4f} ± {np.std(fold_results['auc']):.4f}")

print("\nVARIABLES MÁS IMPORTANTES:")
top_5 = coef_importance.head(5)
for i, (_, row) in enumerate(top_5.iterrows(), 1):
    efecto = "aumenta" if row['Coeficiente'] > 0 else "disminuye"
    print(f"   {i}. {row['Variable']}: {efecto} la probabilidad de Separación de Bienes")

## 8.2 Interpretación de Resultados

### Hallazgos Principales:

1. **Factores que aumentan la probabilidad de elegir Separación de Bienes:**
   - Mayor edad de los contrayentes
   - Mayor nivel educativo
   - Residencia en zonas urbanas
   - Matrimonios tardíos (ambos >35 años)

2. **Factores asociados con Sociedad Conyugal:**
   - Contrayentes más jóvenes
   - Residencia en zonas rurales
   - Menor diferencia de edad entre contrayentes

### Validación del Modelo:

- La **validación cruzada de 5 folds** muestra que el modelo es **estable**, con baja varianza en las métricas entre folds.
- El AUC cercano a 0.6-0.7 indica capacidad predictiva **moderada** - el modelo es mejor que el azar pero tiene limitaciones.
- La consistencia entre métricas de entrenamiento y prueba sugiere **ausencia de sobreajuste**.

### Limitaciones:

1. El régimen matrimonial depende de factores no capturados en los datos (patrimonio previo, asesoría legal, etc.)
2. Posible sesgo de selección: los datos solo incluyen matrimonios registrados formalmente
3. Variables categóricas codificadas de forma ordinal podrían no capturar todas las relaciones

## 8.3 Referencias

- INEGI. (2024). Estadística de Matrimonios (EMAT) 2024.
- Bishop, C. M. (2006). Pattern Recognition and Machine Learning. Springer.
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). The Elements of Statistical Learning.
- Scikit-learn Documentation: https://scikit-learn.org/stable/